In [22]:
#!pip install gitpython

In [298]:
import os
from pathlib import Path
import json
from git import Repo
import datetime

In [306]:
madhava = "@madhava@syft-cache.org"
andrew = "@trask@syft-cache.org"

In [15]:
HERE=!pwd
HERE = Path(HERE[0])
HERE

PosixPath('/Users/madhavajay/dev/syft')

In [18]:
SYFT_HOME = HERE / "test_env"
SYFT_HOME

PosixPath('/Users/madhavajay/dev/syft/test_env')

In [141]:
READ_PERMISSION = "r"
WRITE_PERMISSION = "w"

In [176]:
default_permissions = {READ_PERMISSION:[], WRITE_PERMISSION: []}

In [177]:
def git_repo_path_for_user(user, syft_home=SYFT_HOME):
    return syft_home / user

In [178]:
def git_repo_for_user(user, syft_home=SYFT_HOME):
    git_path = git_repo_path_for_user(user, syft_home=syft_home)
    repo = Repo(git_path)
    return repo

In [207]:
def user_permission_allowed(permissions, user, permission):
    perm = permissions[permission]
    return user in perm or "*" in perm

In [208]:
def validate_perm_file(perm_file_path, user):
    # trigger warning
    with open(perm_file_path, "r") as f:
        perms = default_permissions
        try:
            json_data = json.loads(f.read())
        except Exception as e:
            json_data = perms
        perms.update(json_data)
        
    allowed = user_permission_allowed(perms, user, WRITE_PERMISSION)
    if not allowed:
        file_path = str(perm_file_path).replace(".syftperm", "")
        raise Exception(f"User: {user} does not have {WRITE_PERMISSION} for {file_path}")

In [286]:
def validate_diffs(file_diffs, user_repo, as_user) -> bool:
    # split perm and non perm folder / files from the diff
    permission_file_paths = {}
    actual_file_paths = {}
    
    # build the two dicts
    for diff in file_diffs:
        path = diff.b_path
        if path.endswith(".syftperm"):
            actual = path.replace(".syftperm", "")
            if path not in permission_file_paths:
                permission_file_paths[path] = actual
            if path not in actual_file_paths:
                actual_file_paths[actual] = path
        else:
            perm = path + ".syftperm"
            if path not in permission_file_paths:
                permission_file_paths[perm] = path
            if path not in actual_file_paths:
                actual_file_paths[path] = perm
    user_path = git_repo_path_for_user(user_repo)
    for path in actual_file_paths.keys():
        file_path = user_path / Path(path)
        exists = True if os.path.exists(file_path) else False
        if not exists:
            raise Exception(f"File missing: {file_path}")
        is_dir = True if os.path.isdir(file_path) else False
        name = "folder" if is_dir else "file"
        print(f"Validating {name}: {path}")
        perm_file = actual_file_paths[path]
        perm_file_path = user_path / Path(perm_file)
        if not os.path.exists(perm_file_path):
            raise Exception(f"Missing perm file for user: {path} at: {user_path}/{perm_file}")

        validate_perm_file(perm_file_path, as_user)
    return True

In [299]:
def syt_stage(repo_user, as_user):
    patches_dir = f"./patch_{repo_user}"
    os.makedirs(patches_dir, exist_ok=True)
    repo = git_repo_for_user(repo_user)
    file_diffs = repo.head.commit.diff(None)
    if len(file_diffs) == 0:
        print("Nothing to commit")
        return False

    return validate_diffs(file_diffs, repo_user, as_user)

In [301]:
def syt_commit(repo_user, as_user, message):
    patches_dir = f"./patch_{repo_user}"
    if not syt_stage(repo_user, as_user):
        return

    new_commit = repo.index.commit(message)

    commit_hash = new_commit.hexsha
    print(f"Commit hash: {commit_hash}")

    existing_files = [f for f in os.listdir(patches_dir) if commit_hash in f]
    if existing_files:
        print(f"Patch with the commit hash: {commit_hash} already exists")
        return
    patch_content = repo.git.show(new_commit)

    current_timestamp = int(datetime.datetime.now(datetime.timezone.utc).timestamp())
    patch_file = f"{patches_dir}/{current_timestamp}-{commit_hash}"
    print(f"Writing patch to:{patch_file}")

    with open(patch_file, "w") as f:
        f.write(patch_content)

In [302]:
syt_stage(madhava, madhava)

Validating folder: store/prompts
Validating file: store/prompts/prompts.csv


True

In [303]:
syt_commit(madhava, madhava, "Adding store files")

Validating folder: store/prompts
Validating file: store/prompts/prompts.csv
Commit hash: 973cd7ca6e501f9953ea3ed699cbcdb5c4db74af
Writing patch to:./patch_@madhava@syft-cache.org/1723768647-973cd7ca6e501f9953ea3ed699cbcdb5c4db74af


In [310]:
syt_stage(andrew, andrew)

Validating folder: projects/my_first_project/chat_madhava_trask
Validating file: projects/my_first_project/chat_madhava_trask/msg1.txt


True

In [314]:
syt_commit(andrew, andrew, "Added chat with Madhava")

Validating folder: projects/my_first_project/chat_madhava_trask
Validating file: projects/my_first_project/chat_madhava_trask/msg1.txt
Commit hash: 65499dcdfe2bf4bcb204affb65a174e9aef2bc7c
Writing patch to:./patch_@trask@syft-cache.org/1723769252-65499dcdfe2bf4bcb204affb65a174e9aef2bc7c


In [304]:
# import hashlib

# def simulate_commit_hash(repo, commit_message):
#     # Get the tree hash of the staged changes
#     tree_hash = repo.index.write_tree()

#     # Get the parent commit hash (usually the latest commit)
#     parent_commit = repo.head.commit.hexsha

#     # Prepare the commit content
#     commit_content = f"tree {tree_hash}\n"
#     commit_content += f"parent {parent_commit}\n"
#     commit_content += f"author {repo.config_reader().get_value('user', 'name')} <{repo.config_reader().get_value('user', 'email')}> {int(datetime.datetime.now().timestamp())} +0000\n"
#     commit_content += f"committer {repo.config_reader().get_value('user', 'name')} <{repo.config_reader().get_value('user', 'email')}> {int(datetime.datetime.now().timestamp())} +0000\n\n"
#     commit_content += f"{commit_message}\n"

#     # Calculate the commit hash
#     commit_hash = hashlib.sha1(f"commit {len(commit_content)}\0{commit_content}".encode()).hexdigest()

#     return commit_hash